### Activation Functions
 - Sigmoid
   - torch.nn.sigmoid(x)
 - tanh
   - torch.nn.tanh(x)
 - ReLU
   - torch.nn.relu(x)
 - LeakyReLU
   - torch.nn.leaky_relu(x, alpha)

### Optimizers in Pytorch
 - torch.optim.SGD
 - torch.optim.Adadelta
 - torch.optim.Adagrad
 - torch.optim.Adam
 - torch.optim.SparseAdam
 - torch.optim.Adamax
 - torch.optim.ASGD
 - torch.optim.LBFGS
 - torch.optim.Rprop

![Optimizer](./Optimizer.png)

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear = nn.Linear(784, 10, bias=True).to(device)
        nn.init.normal_(self.linear.weight)
        
    def forward(self, x):
        return self.linear(x)

In [9]:
model = Model()
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
from IPython.display import clear_output


total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for batch_num, sample in enumerate(data_loader):
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        x, y = sample
        x = x.view(-1, 28 * 28).to(device)
        y = y.to(device)
        
        hypothesis = model(x)
        cost = criterion(hypothesis, y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    clear_output(wait=True)
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0015 cost = 0.388621062
Learning finished


In [13]:
# Test the model using test sets
with torch.no_grad():
    x_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)

    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8848000168800354
Label:  8
Prediction:  8


/Users/honeyeob/python_workspace/Pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/honeyeob/python_workspace/Pytorch/lib/python3.7/site-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
